In [1]:
# TODO
#!{__import__('sys').executable} -m pip install --quiet openstudio

In [2]:
import contextlib
import builtins

@contextlib.contextmanager
def temporary_attr(o, name: str):
    a = builtins.getattr(o, name)
    try: yield
    finally: builtins.setattr(o, name, a)

import os
import sys

@contextlib.contextmanager
def temporary_search_path(*paths):
    with temporary_attr(sys, 'path'):
        setattr(sys, 'path', [str(p) for p in paths])
        try: yield
        finally: pass

In [3]:
import os
import shutil

def find_energyplus():
    return os.path.dirname(
        os.path.realpath(shutil.which('energyplus'))
    )

In [4]:
# TODO
energyplus_path = os.path.expanduser('~/.local/EnergyPlus-23-1-0')
with temporary_search_path(energyplus_path):
    import pyenergyplus as eplus
    import pyenergyplus.api

In [52]:
import packaging
import io
import csv
import collections


class EnergyPlusEMS:
    _ep_api = eplus.api.EnergyPlusAPI()
    assert (
        packaging.version.Version(_ep_api.api_version()) 
            >= packaging.version.Version('0.2')
    )

    def __init__(self):
        pass
    
    def __enter__(self):
        if getattr(self, '_ep_state', None) is not None:
            raise Exception()
        self._ep_state = self._ep_api.state_manager.new_state()
        return self
        
    def __exit__(self, *_exc_args):
        if getattr(self, '_ep_state', None) is None:
            raise Exception()
        self._ep_api.state_manager.delete_state(self._ep_state)

    def _exec(self, *args):
        self._ep_api.state_manager.reset_state(self._ep_state)
        return self._ep_api.runtime.run_energyplus(
            self._ep_state,
            command_line_args=args
        )

    def _available_data(self):
        with io.StringIO(
            self._ep_api.exchange
                .list_available_api_data_csv(self._ep_state)
                .decode()
        ) as f:
            def _ep_csv_reader(f, default_title=None):
                title = default_title
                for row in csv.reader(f):
                    if len(row) == 1:
                        title = row.pop()
                    yield title, row
        
            d = collections.defaultdict(lambda: [])
            for title, row in _ep_csv_reader(f):
                if not row:
                    continue
                d[title].append(row)
        
            colnames = {
                '**ACTUATORS**': ['Type', 'ComponentTypeName', 'ControlTypeName', 'UniqueIDName'],
                '**INTERNAL_VARIABLES**': ['Type', 'DataTypeName', 'UniqueIDName'],
                '**PLUGIN_GLOBAL_VARIABLES**': ['Type', 'Name'],
                '**TRENDS**': ['Type', 'Name'],
                '**METERS**': ['Type', 'Name'],
                '**VARIABLES**': ['Type', 'VarNameOnly', 'KeyNameOnlyUC']
            }
 
            return {title: pd.DataFrame(d[title], columns=colnames[title]) for title in d}
        

In [54]:
with EnergyPlusEMS() as ep_ems:
    ep_ems._exec(
        # TODO
        '--output-directory', 'build/demo-eplus',
        '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
        f'{energyplus_path}/ExampleFiles/CoolingTower_VariableSpeed_MultiCell.idf'
    )
    #ep_ems._ep_api.runtime.callback

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.12 01:39
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "INT-FLOOR-TOPSIDE"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "IEAD_R-15 CI_ROOF"
Calculating CTFs for "STANDARD_INT-WALL"
Calculating CTFs for "MASS_R-7.6 CI_EXT-WALL"
Calculating CTFs for "UNHEATED - 4IN SLAB WITH CARPET_EXT-SLAB"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorpt

EnergyPlus Completed Successfully.


In [55]:
import io
import pandas as pd


ep_ems = EnergyPlusEMS().__enter__()

#ep_ems._exec('--help')
ep_ems._exec(
    # TODO
    '--design-day',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{energyplus_path}/ExampleFiles/CoolingTower_VariableSpeed_MultiCell.idf'
)


EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.12 01:39
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "INT-FLOOR-TOPSIDE"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "IEAD_R-15 CI_ROOF"
Calculating CTFs for "STANDARD_INT-WALL"
Calculating CTFs for "MASS_R-7.6 CI_EXT-WALL"
Calculating CTFs for "UNHEATED - 4IN SLAB WITH CARPET_EXT-SLAB"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorpt

EnergyPlus Completed Successfully.


0

In [56]:
df_available_data = ep_ems._available_data()

df_available_data['**INTERNAL_VARIABLES**']

,Type,DataTypeName,UniqueIDName
0,InternalVariable,Plant Design Volume Flow Rate,SWHSYS1
1,InternalVariable,Plant Design Volume Flow Rate,HEATSYS1
2,InternalVariable,Plant Design Volume Flow Rate,COOLSYS1
3,InternalVariable,Plant Design Volume Flow Rate,TOWERWATERSYS


In [58]:
df_available_data.get('**ACTUATORS**')

In [ ]:
ep_ems.__close__()
